In [ ]:
!pip install opendatasets

In [ ]:
import opendatasets as od

In [ ]:
url = 'https://www.kaggle.com/datasets/mahmoudreda55/satellite-image-classification'
od.download(url)

In [ ]:
train_path = r'/content/satellite-image-classification/data'

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
img = mpimg.imread('/content/satellite-image-classification/data/cloudy/train_10021.jpg')
print(img)

In [ ]:
imgplot = plt.imshow(img)

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.models import Model
from keras import optimizers, losses, activations, models
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.applications import InceptionV3

base_model = InceptionV3(weights='imagenet', 
                                include_top=False, 
                                input_shape=(150, 150,3))
base_model.trainable = False

add_model = Sequential()
add_model.add(base_model)
add_model.add(GlobalAveragePooling2D())
add_model.add(Dropout(0.2))
add_model.add(Dense(4, 
                    activation='softmax'))

model = add_model
model.compile(loss='categorical_crossentropy', 
              optimizer=tf.keras.optimizers.SGD(learning_rate=1e-4, 
              momentum=0.9),
              metrics=['accuracy'])
model.summary()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import preprocess_input

#Normalize the data

#Using Augmentaion
image_datagen = ImageDataGenerator(rescale=1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest', 
      validation_split=0.2,)


train_dataset = image_datagen.flow_from_directory(batch_size=32,
                                                 directory='/content/satellite-image-classification/data',
                                                 shuffle=True,
                                                 target_size=(150, 150), 
                                                 subset="training",
                                                 class_mode='categorical')

validation_dataset = image_datagen.flow_from_directory(batch_size=32,
                                                 directory='/content/satellite-image-classification/data',
                                                 shuffle=True,
                                                 target_size=(150, 150), 
                                                 subset="validation",
                                                 class_mode='categorical')

In [ ]:
history = model.fit(
            train_dataset,
            epochs=20,
            validation_data=validation_dataset,
            )

In [ ]:
import matplotlib.pyplot as plt

# Plot the model results
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
model.save('satelite_image_classification.h5')

In [ ]:
cloudy_path = r'/content/satellite-image-classification/data/cloudy'
desert_path = r'/content/satellite-image-classification/data/desert'
green_area_path = r'/content/satellite-image-classification/data/green_area'
water_path = r'/content/satellite-image-classification/data/water'

In [ ]:
import cv2
import os

In [ ]:
images = []
labels = []

In [ ]:
for i in os.listdir(cloudy_path):
  img = cv2.imread(cloudy_path + '/' + i, 0)
  img = cv2.resize(img, (64,64))
  img = img/255
  img = img.flatten()
  images.append(img)
  labels.append(0)

for i in os.listdir(desert_path):
  img = cv2.imread(desert_path + '/' + i, 0)
  img = cv2.resize(img, (64,64))
  img = img/255
  img = img.flatten()
  images.append(img)
  labels.append(1)

for i in os.listdir(green_area_path):
  img = cv2.imread(green_area_path + '/' + i, 0)
  img = cv2.resize(img, (64,64))
  img = img/255
  img = img.flatten()
  images.append(img)
  labels.append(2)

for i in os.listdir(water_path):
  img = cv2.imread(water_path + '/' + i, 0)
  img = cv2.resize(img, (64,64))
  img = img/255
  img = img.flatten()
  images.append(img)
  labels.append(3)


In [ ]:
images[0]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size = 0.2)

In [ ]:
from sklearn.svm import SVC

In [ ]:
clf = SVC()
clf.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score

preds = clf.predict(X_test)

In [ ]:
accuracy_score(preds, y_test)

In [ ]:
import joblib

In [ ]:
joblib.dump(clf, 'classifier.joblib')